In [147]:
import pandas as pd
import numpy as np
import time
import datetime
import chromedriver_autoinstaller
import re
import requests
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from selenium import webdriver
from tqdm import tqdm
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver import ActionChains

chrome_ver = chromedriver_autoinstaller.get_chrome_version().split('.')[0]  #크롬드라이버 버전 확인
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--incognito") # 시크릿 모드
chrome_options.add_argument("--no-sandbox") # 리소스에 대한 엑서스 방지
chrome_options.add_argument("--disable-setuid-sandbox") # 크롬 충돌 방지
chrome_options.add_argument("--disable-dev-shm-usage") # 메모리 부족 에러 방지
chrome_options.add_experimental_option('excludeSwitches', ['enable-logging'])
try: # 크롬 드라이버
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)   
except:
    chromedriver_autoinstaller.install(True)
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

base_url = "https://au.investing.com/equities/vietnam"
driver.get(base_url)
driver.maximize_window()
time.sleep(3)
driver.find_element(By.XPATH, '//*[@id="__next"]/header/div[1]/section/div[3]/ul/li[1]').click()
time.sleep(1)
driver.find_element(By.XPATH, '/html/body/div[2]/div/div[2]/div/form/button[3]').click()
id = "syyoo@hebronstar.com"
pw = "tlsdud12!!"
time.sleep(1)
driver.find_element(By.XPATH, "/html/body/div[2]/div/div[2]/div/form/div[3]/input").send_keys(id)
driver.find_element(By.XPATH, "/html/body/div[2]/div/div[2]/div/form/div[5]/input").send_keys(pw)
driver.find_element(By.XPATH, "/html/body/div[2]/div/div[2]/div/form/button").send_keys("\n")
time.sleep(10)
select_box = driver.find_element(By.XPATH,'//*[@id="index-select"]').click()
select_all_stock = driver.find_element(By.XPATH,'//*[@id="index-select"]/div[2]/div/div/div[1]').click()
time.sleep(5)
SCROLL_PAUSE_SEC = 3

# 스크롤 높이 가져옴
last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    
    try:
        some_tag = driver.find_element(By.XPATH, '//*[@id="__next"]/div[2]/div/div/div[2]/main/div[3]/div[2]/div')
        action = ActionChains(driver)
        action.move_to_element(some_tag).perform()
        some_tag.click()
    except NoSuchElementException: break
    # 끝까지 스크롤 다운
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # SCROLL_PAUSE_SEC만큼 대기
    time.sleep(SCROLL_PAUSE_SEC)
    
    # 스크롤 다운 후 스크롤 높이 다시 가져옴
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height
soup = BeautifulSoup(driver.page_source,"html.parser")
div_element = soup.find('div', {'data-test': 'dynamic-table'})
all_atag_maintable = div_element.find_all("a")
company_links = []


In [148]:
def download_historial(self):
    for a in all_atag_maintable:
        company_link = a.attrs["href"] #find로 한번더 돌려준다.
        company_links.append(company_link)
    company_table_css = driver.find_element(By.CSS_SELECTOR, 'div[data-test="dynamic-table"]')
    company_table_html = company_table_css.get_attribute('outerHTML')
    company_table = pd.read_html(company_table_html)[0]
    stock_history_company_links = [i+"-historical-data" for i in company_links]
    for idx, name in tqdm(zip(stock_history_company_links[912:], company_table["Name"][912:])):
        driver.get("https://au.investing.com/" + idx)
        time.sleep(2)
        try: click_time = driver.find_element(By.XPATH, '//*[@id="history-timeframe-selector"]').click()
        except NoSuchElementException:
            driver.get("https://au.investing.com/" + idx)
            click_time = driver.find_element(By.XPATH, '//*[@id="history-timeframe-selector"]').click()
        click_month = driver.find_element(By.XPATH, '//*[@id="react-select-2-option-1"]').click()
        try: historical_data = driver.find_element(By.XPATH, '//*[@id="__next"]/div[2]/div/div/div[2]/main/div/div[6]/div/div/div[2]/div[2]')
        except NoSuchElementException: 
            try:
                historical_data = driver.find_element(By.XPATH, '//*[@id="__next"]/div[2]/div/div/div[2]/main/div/div[7]/div/div/div[2]/div[2]')
            except: 
                driver.refresh()
                click_time = driver.find_element(By.XPATH, '//*[@id="history-timeframe-selector"]').click()
                click_month = driver.find_element(By.XPATH, '//*[@id="react-select-2-option-1"]').click()
                try: historical_data = driver.find_element(By.XPATH, '//*[@id="__next"]/div[2]/div/div/div[2]/main/div/div[6]/div/div/div[2]/div[2]')
                except NoSuchElementException: historical_data = driver.find_element(By.XPATH, '//*[@id="__next"]/div[2]/div/div/div[2]/main/div/div[7]/div/div/div[2]/div[2]')
        time.sleep(2)
        historical_data.click()
        input_element = historical_data.find_element(By.TAG_NAME, 'input')
        while input_element.get_attribute('value') != "2018-01-01":
            input_element.clear()
            input_element.send_keys("20180101")
        month_button = historical_data.find_element(By.TAG_NAME, 'button')
        month_button.send_keys("\n")
        time.sleep(3)
        a = historical_data.find_element(By.TAG_NAME, 'a')
        a.click()

353it [1:01:22, 10.43s/it]


In [149]:
stock_history_company_links

['/equities/an-phat.-jsc-historical-data',
 '/equities/asia-commercial-bank-historical-data',
 '/equities/adc-historical-data',
 '/equities/tan-binh-culture-co.-historical-data',
 '/equities/vam-historical-data',
 '/equities/asia-mineral-historical-data',
 '/equities/alphanam-m-e-historical-data',
 '/equities/amvibiotech-historical-data',
 '/equities/apsi-historical-data',
 '/equities/apec-investmen-historical-data',
 '/equities/apec-securitie-historical-data',
 '/equities/airimex-jsc-historical-data',
 '/equities/wmt-sana-joint-historical-data',
 '/equities/jsc-482-historical-data',
 '/equities/but-son-cement-packing-jsc-historical-data',
 '/equities/bim-son-cement-jsc-historical-data',
 '/equities/bidibook-historical-data',
 '/equities/danangbook-historical-data',
 '/equities/biconco-historical-data',
 '/equities/bach-dang-tmc-historical-data',
 '/equities/bamcorp.-historical-data',
 '/equities/bac-lieu-fis-historical-data',
 '/equities/bim-son-packing-jsc-historical-data',
 '/equiti